<a href="https://colab.research.google.com/github/akshaygrao77/MABCricketProject/blob/main/Template_Cricket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import math
import random
from operator import attrgetter
import json
import statistics
import itertools
from itertools import product
np.random.seed(0)
# Students will submit their files with their team-name.py 
# Student have to use the Team as their parent class

In [ ]:
class State:
  def __init__(self,ball,total_runs,wickets_left,feature_batter,feature_bowler,current_batters_list,current_bowlers_list,batting_action=None,bowling_action=None):
    self.feature_batter = np.copy(feature_batter)
    self.feature_bowler = np.copy(feature_bowler)
    self.num_ball = ball
    self.total_runs = total_runs
    self.wickets_left = wickets_left
    self.bowling_action = bowling_action
    self.batting_action = batting_action
    self.current_bowlers_list = np.copy(current_bowlers_list)
    self.current_batters_list = np.copy(current_batters_list)
  
  def __eq__(self, other) : 
    # print("self: "+str(self))
    # print("other: "+str(other))
    check1 = np.array_equal(self.feature_batter,other.feature_batter)
    # print("check1:",check1)
    check2 = np.array_equal(self.feature_bowler,other.feature_bowler)
    # print("check2:",check2)
    check3 = (self.num_ball == other.num_ball)
    # print("check3:",check3)
    check4 = (self.total_runs == other.total_runs)
    # print("check4:",check4)
    check5 = (self.wickets_left == other.wickets_left)
    # print("check5:",check5)
    check6 = (self.bowling_action == other.bowling_action)
    # print("check6:",check6)
    check7 = (self.batting_action == other.batting_action)
    # print("check7:",check7)
    check8 = np.array_equal(self.current_bowlers_list,other.current_bowlers_list)
    # print("check8:",check8)
    check9 = np.array_equal(self.current_batters_list,other.current_batters_list)
    # print("check9:",check9)

    return ( check1 and check2 and check3 and check4 and check5 and check6 and check7 and check8 and check9)
  
  def __str__(self):
    return "feature_batter: "+str(self.feature_batter) + "\n feature_bowler: "+str(self.feature_bowler)+"\n num_ball: "+str(self.num_ball)+" total_runs: "+str(self.total_runs)+" wickets_left: "+str(self.wickets_left)+"\n bowling_action: "+str(self.bowling_action)+"\n batting_action: "+str(self.batting_action)+" \n current_bowlers_list: "+str(self.current_bowlers_list)+"\n current_batters_list: "+str(self.current_batters_list)

In [ ]:
class GameConfig:
  def __init__(self,horizon_balls,iterative_deepening_depth=1,ucb_mode=None,num_rand_rollout_multiplier=None):
    self.horizon_balls = horizon_balls
    self.ucb_mode = ucb_mode
    self.num_rand_rollout_multiplier = num_rand_rollout_multiplier
    self.iterative_deepening_depth = iterative_deepening_depth

In [ ]:
class MCTSNode:
  def __init__(self,parent,to_play,state,isNodeStochastic = False):
    self.visit_count = 0
    self.to_play = to_play
    # Dictionary of action,node array (bcoz sometimes, same action can lead to more than one node in case of stochastic actions)
    self.children = []
    self.state = state
    self.isNodeStochastic = False
    self.parent = parent
    # When a state is stochastic
    self.nodeOccurenceProbability = 0
    self.valueOfNode = 0
    self.ucb_scoreOfNode = 0
  
  def expand_children(self,gameLogic):
     
    children_nodes = []
    possible_actions = gameLogic.get_possible_actions_in_state(self.to_play,self.state)
    children_states = gameLogic.get_children_states(self.to_play,self.state,possible_actions)
    for each_child_state in children_states:
      next_to_play = gameLogic.get_next_to_play_order(self.to_play)
      child_node = MCTSNode(parent=self,to_play=next_to_play,state=each_child_state)
      if(next_to_play == gameLogic.stochastic_role_to_play):
        child_node.isNodeStochastic = True
      children_nodes.append(child_node)
    self.children = children_nodes
  
  def expand_stochastic_nodes(self,gameLogic):
    # print("$$$$$$$$$$$$$$$$$$$ Stochastic expand $$$$$$$$$$$$$$$$$$$$$$")
    wicket,runs = gameLogic.environment.get_outcome(self.state.feature_batter, self.state.feature_bowler, self.state.batting_action, self.state.bowling_action)
    new_state = gameLogic.get_next_reward_state(wicket,runs,self.state)
    for each_child in self.children:
      # print("Each child state: "+str(each_child.state.total_runs)+" \n New state: "+str(new_state.total_runs))
      if(each_child.state == new_state):
        each_child.visit_count += 1
        each_child.parent.update_node_probability()
        return each_child
    
    next_to_play = gameLogic.get_next_to_play_order(self.to_play)
    child_node = MCTSNode(parent=self,to_play=next_to_play,state=new_state)
    child_node.visit_count = 1

    child_node.parent.update_node_probability()
    
    self.children.append(child_node)

    return child_node
  
  def update_node_probability(self):
    for each_child in self.children:
      each_child.nodeOccurenceProbability = each_child.visit_count / each_child.parent.visit_count

  def select_best_child(self,gameLogic,round_num=None):
    if(round_num is None):
      round_number = self.parent.visit_count
    else:
      round_number = round_num

    for i in range(len(self.children)):
      if(self.children[i].visit_count == 0):
        self.children[i].visit_count = 1
        if(self.children[i].isNodeStochastic):
          return self.children[i].expand_stochastic_nodes(gameLogic)
        return self.children[i]
    for each_child_node in self.children:
      each_child_node.compute_ucb_score(round_number,gameLogic)
    best_child =  max(self.children , key = attrgetter('ucb_scoreOfNode'))
    best_child.visit_count = best_child.visit_count + 1
    if(best_child.isNodeStochastic):
      return best_child.expand_stochastic_nodes(gameLogic)
    return best_child


  def compute_ucb_score(self,round_number,gameLogic):
    # All the modes are just for comparision currently. Only best one will be fixed later
    if(gameLogic.game_config.ucb_mode is None):
      # Tighter bound version of UCB
      exploration_term = math.sqrt(2 * math.log(1 + round_number * math.pow(math.log(round_number),2)) / self.visit_count)
    elif(gameLogic.game_config.ucb_mode == 'VERYLOOSE'):
      # Another very loose bound version for UCB
      exploration_term = math.sqrt(4 * math.log(round_number) / self.visit_count)  
    elif(gameLogic.game_config.ucb_mode == 'LOOSE'):
      # Another looser bound version for UCB
      exploration_term = math.sqrt(2 * math.log(round_number) / self.visit_count)  
    elif(gameLogic.game_config.ucb_mode == 'GREEDY'):
      # Greedy choice means no UCB concept used at all
      exploration_term = 0

    if(self.to_play == 'me_to_play'):
      self.ucb_scoreOfNode = self.valueOfNode + exploration_term
    elif(self.to_play == 'opp_to_play'):
      self.ucb_scoreOfNode = self.valueOfNode + exploration_term
  
  def simulate_random_rollout(self,gameLogic):
    if(gameLogic.is_state_terminal(self.state)):
      return self.state.wickets_left,self.state.total_runs
    
    if(len(self.state.feature_batter.shape)>1):
      print("Simulation Invalid shape:"+str(self.state.feature_batter) +" shaaappppeee: "+str(self.state.feature_batter.shape)+" previous batters:"+str(self.state.current_batters_list))
      raise Exception("Simulation Invalid shape")
    
    if(self.to_play != 'chance_reward'):
      next_to_play = gameLogic.get_next_to_play_order(self.to_play)
      possible_actions = gameLogic.get_possible_actions_in_state(self.to_play,self.state)
      random_action = random.choice(possible_actions)
      next_child_state = gameLogic.get_each_children_state(self.to_play,self.state,random_action)
      if(gameLogic.is_state_terminal(next_child_state)):
        return next_child_state.wickets_left,next_child_state.total_runs
      next_child_node = MCTSNode(parent=self,to_play=next_to_play,state=next_child_state)
      return next_child_node.simulate_random_rollout(gameLogic)


    number_of_sampling_per_call = 11
    sum_of_runs = 0
    sum_of_wickets = 0
    random_batting_action = random.choice(gameLogic.batting_actions)
    random_bowling_action = random.choice(gameLogic.bowling_actions)
    for i in range(number_of_sampling_per_call):
      wicket,run = gameLogic.environment.get_outcome(self.state.feature_batter, self.state.feature_bowler, random_batting_action, random_bowling_action)
      sum_of_wickets+= wicket
      sum_of_runs += run
    average_runs = sum_of_runs / number_of_sampling_per_call
    average_wicket = sum_of_wickets / number_of_sampling_per_call
    if(average_wicket > 0.5):
      wicket = 1
    else:
      wicket = 0
    
    new_state = gameLogic.get_next_reward_state(wicket,average_runs,self.state)
    
    if(gameLogic.is_state_terminal(new_state)):
      return new_state.wickets_left,new_state.total_runs
    next_child_node = MCTSNode(parent=self,to_play=self.to_play,state=new_state)
    return next_child_node.simulate_random_rollout(gameLogic)
  
  def back_propogate_node_value(self,gameLogic,v_list=None):
    if(v_list is None):
      value_list = []
    else:
      value_list = v_list

    if(self.parent is None):
      return value_list
    
    if(self.parent.isNodeStochastic):
      all_children = self.parent.children
      expected_value = 0
      prob_sum_for_sanity_check = 0
      for each_children in all_children:
        prob_sum_for_sanity_check += each_children.nodeOccurenceProbability
        expected_value += (each_children.nodeOccurenceProbability * each_children.valueOfNode)
      
      # Purely sanity check code to catch issues. Check is whether sum of probs = 1
      if((prob_sum_for_sanity_check - 1) > 0.001):
        print("sum of probs: "+str(prob_sum_for_sanity_check))
        print("nodeOccurenceProbability doesn't sum to 1 for parent object: "+str(self.parent))
        raise Exception("nodeOccurenceProbability doesn't sum to 1")
      
      self.parent.update_value_of_node(expected_value)
      value_list.append(expected_value)
    else:
      self.parent.update_value_of_node(self.valueOfNode)
      value_list.append(self.valueOfNode)

    return self.parent.back_propogate_node_value(gameLogic,value_list)
    
      
  # Phase 2 needs shift(maybe)
  def update_value_of_node(self,value):
    if(math.isinf(value)):
      print("Error at object: "+str(self))
      raise Exception("Value is infinity. Bug exists")
    # Value is always positive because we want to maximize value (hence runs scored too)
    if(self.to_play == 'me_to_play' or self.to_play == 'chance_reward'):
      if(value > 0):
        self.valueOfNode = self.update_mean(self.valueOfNode,-value,self.visit_count) 
      else:
        self.valueOfNode = self.update_mean(self.valueOfNode,value,self.visit_count) 
    # Value is always negative because we want to maximize value (minimize runs scored)
    elif(self.to_play == 'opp_to_play'):
      if(value < 0):
        self.valueOfNode = self.update_mean(self.valueOfNode,-value,self.visit_count) 
      else:
        self.valueOfNode = self.update_mean(self.valueOfNode,value,self.visit_count) 

  def update_mean(self,current_mean, new_value, n ):
    """ calculate the new mean from the previous mean and the new value """
    ret = (1 - 1.0/n) * current_mean + (1.0/n) * new_value
    if(math.isinf(ret)):
      print("Error. current_mean:"+str(current_mean)+" new_value:"+str(new_value)+" n:"+str(n))
      raise Exception("Value is infinity. Bug exists")
    return ret

  def is_node_expanded(self):
    if(len(self.children)==0):
      return False
    return True
  
  def __eq__(self, other) : 
    return self.__dict__ == other.__dict__

  def __str__(self):
    return "\n Visit_count: "+str(self.visit_count)+" to_play:"+str(self.to_play)+" isNodeStochastic: "+str(self.isNodeStochastic)+" valueOfNode:"+str(self.valueOfNode)+" nodeOccurenceProbability: "+str(self.nodeOccurenceProbability)+" ucb_scoreOfNode:"+str(self.ucb_scoreOfNode)+" \n state:"+str(self.state)
    

In [ ]:
class GameLogic:

  def __init__(self,environment,team,game_config):
    self.game_config = game_config
    self.my_team = team
    self.environment = environment
    self.to_play_order = np.array(('me_to_play','opp_to_play','chance_reward'))
    self.stochastic_role_to_play = 'chance_reward'
    self.batting_actions = [0,1,2,3,4,6]
    self.bowling_actions = ['e','n','a']

  def is_state_terminal(self,given_state):
    if(given_state.num_ball >= self.game_config.horizon_balls):
      return True
    if(given_state.wickets_left == 0):
      return True
    return False

  def get_next_reward_state(self,wicket,runs,previous_state):
    if(previous_state.wickets_left == 0):
      print("Invalid state of batters_list:"+str(previous_state.current_batters_list))
      raise Exception("Invalid state of batters_list:"+str(previous_state.current_batters_list))
    
    nextState = State(total_runs = previous_state.total_runs , feature_batter = previous_state.feature_batter,feature_bowler = previous_state.feature_bowler,ball = (previous_state.num_ball + 1),wickets_left = previous_state.wickets_left,current_batters_list=previous_state.current_batters_list,current_bowlers_list = previous_state.current_bowlers_list)
    if(wicket > 0):
      nextState.wickets_left = nextState.wickets_left - 1
      if(nextState.wickets_left > 0):
        batter_index = self.my_team.get_next_batter(previous_state.current_batters_list)
        previous_batter_index = np.where(previous_state.current_batters_list == -1)[0][0]
        nextState.current_batters_list[previous_batter_index] = 0
        nextState.current_batters_list[batter_index] = -1
        nextState.feature_batter = self.my_team.self_features[batter_index,0:2]

    nextState.total_runs = nextState.total_runs + runs
    
    if ((nextState.num_ball) % 6 == 0 ):
      # print("get_next_reward_state current_bowlers_list:"+str(nextState.current_bowlers_list))
      previous_bowler_index = np.where(nextState.current_bowlers_list < 0)[0][0]
      nextState.current_bowlers_list[previous_bowler_index] = -(nextState.current_bowlers_list[previous_bowler_index] + 1)
      
      ind_array =  np.where(nextState.current_bowlers_list > 0)[0]
      if(ind_array.shape[0] != 0):
        bowler_index = ind_array[0]
        nextState.current_bowlers_list[bowler_index] = -(nextState.current_bowlers_list[bowler_index])
        nextState.feature_bowler = self.my_team.opponent_features[bowler_index,2:4]
    
    
    if(len(nextState.feature_batter.shape)>1):
      print("Invalid shape:"+str(nextState.feature_batter) +" shaaappppeee: "+str(nextState.feature_batter.shape)+"  batter_index:"+str(batter_index)+" previous batters:"+str(previous_state.current_batters_list))
      raise Exception("Invalid shape")

    return nextState


  def get_next_to_play_order(self,current_to_play):
    next_index = np.where(self.to_play_order == current_to_play )[0][0] + 1

    if(next_index == len(self.to_play_order)):
      return self.to_play_order[0]
    return self.to_play_order[next_index]

  def get_possible_actions_in_state(self,to_play,state):
    if(to_play == 'me_to_play'):
      return self.batting_actions
    elif(to_play == 'opp_to_play'):
      return self.bowling_actions

  def get_children_states(self,to_play,state,possible_actions):
    children_states = []
    for each_action in possible_actions:
      child_state = self.get_each_children_state(to_play,state,each_action)
      children_states.append(child_state)
    return children_states
  
  def get_each_children_state(self,to_play,state,each_action):
    if(to_play == 'me_to_play'):
      child_state = State(total_runs= state.total_runs,feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,ball=state.num_ball,wickets_left=state.wickets_left,current_bowlers_list=state.current_bowlers_list,current_batters_list=state.current_batters_list,batting_action=each_action)
      return child_state
    elif(to_play == 'opp_to_play'):
      child_state = State(total_runs = state.total_runs,feature_batter=state.feature_batter,feature_bowler=state.feature_bowler,ball=state.num_ball,wickets_left=state.wickets_left,current_bowlers_list=state.current_bowlers_list,current_batters_list = state.current_batters_list,batting_action=state.batting_action,bowling_action=each_action)
      return child_state
      
  

In [ ]:
# If true continue
def within_time_constraints(start_time,time_taken_for_loop,action_timeout):
  current_time =  time. time()
  time_left = (action_timeout - (current_time -start_time))
  if(time_left > (0.1)):
    return True
  return False

In [ ]:
class MCTSAlgo:
  def __init__(self,environment,team,game_config):
    self.environment = environment;
    self.my_team = team
    self.game_config = game_config

  def run(self,state,overall_action_timeout,my_role):
    overall_start_time =  time. time()
    gameLogic = GameLogic(self.environment,self.my_team,self.game_config)
    root = MCTSNode(None,'me_to_play',state)
    root.expand_children(gameLogic)
    current_time =  time. time()
    number_of_iterative_deepening = self.game_config.iterative_deepening_depth
    actual_horizon = self.game_config.horizon_balls
    horizon_per_ID = []
    horizon_increase_size = actual_horizon//number_of_iterative_deepening
    for i in range(number_of_iterative_deepening):
      if(i== number_of_iterative_deepening - 1):
        horizon_per_ID.append(actual_horizon)
      else:
        current_horizon = (i+1)*horizon_increase_size
        if(state.num_ball < current_horizon):
          horizon_per_ID.append(current_horizon)
    
    action_timeout = overall_action_timeout/len(horizon_per_ID)
    iterative_depth_count = 0
    time_left = (overall_action_timeout - (current_time -overall_start_time))
    time_taken_for_loop = 0.0000001
    # If time left is less than twice the time for previous simulation, then exit
    while(True):
      if(iterative_depth_count == len(horizon_per_ID)):
        break
      elif(iterative_depth_count == (len(horizon_per_ID)- 1)):
        current_time =  time. time()
        action_timeout = (overall_action_timeout - (current_time - overall_start_time))
      simulation_count = 0
      current_sim_depth = 0
      all_simulation_depth = []

      if(not(within_time_constraints(overall_start_time,time_taken_for_loop,overall_action_timeout))):
        break
      
      current_horizon = horizon_per_ID[iterative_depth_count]
      self.game_config.horizon_balls = current_horizon
      
      start_time =  time. time()
      iterative_depth_count+=1
      print("In iterative deepening round number ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"+str(iterative_depth_count)+"time_taken_for_loop: "+str(time_taken_for_loop)+" Horizon ID: "+str(horizon_per_ID)+" Actiontimeout: "+str(action_timeout))
      while(True):
        if(not(within_time_constraints(start_time,time_taken_for_loop,action_timeout))):
          break

        simulation_count += 1
        # print("*********************************************")
        # print("Simulation count:",simulation_count)
        start_of_loop_time = time.time()

        next_best_child = root
        next_best_child.visit_count += 1
        current_sim_depth = 1
        # Selection phase
        while(next_best_child.is_node_expanded() and not(gameLogic.is_state_terminal(next_best_child.state))):
          current_sim_depth += 1
          if(next_best_child.parent is None):
            next_best_child = next_best_child.select_best_child(gameLogic,round_num=simulation_count)
          else:
            next_best_child = next_best_child.select_best_child(gameLogic)
          # print("!!!!!!!!!!!!!!!!!!!!!!!!! Selection phase !!!!!!!!!!!!!!!!!!!!!"+str(next_best_child))
        
        # print("Selection phase complete. Best child",next_best_child)
        if(not(within_time_constraints(start_time,time_taken_for_loop,action_timeout))):
          simulation_count -= 1
          break

        #Expansion phase
        next_best_child.expand_children(gameLogic)
        
        # Simulation phase
        # Random rollout done only once is bad since among huge state space only one path is sampled for value at horizon.
        # So one heuristic is to increase the number of rollouts when depth of selection increases.(intuition is when depth increases MCTS is confident about that path. Hence explore that more and get actual value)
        number_of_random_rollouts = 2
        if(self.game_config.num_rand_rollout_multiplier is None):
          number_of_random_rollouts = 1 + current_sim_depth * math.floor(action_timeout)
        else:
          number_of_random_rollouts = 1 + current_sim_depth * self.game_config.num_rand_rollout_multiplier
        
        sum_runs_horizon = 0
        for i in range(number_of_random_rollouts):
          wicket_left_at_horizon, runs_at_horizon = next_best_child.simulate_random_rollout(gameLogic)
          sum_runs_horizon += runs_at_horizon
        
        avg_runs_horizon = sum_runs_horizon / number_of_random_rollouts
        # Calculate the run per ball and use that as value of node
        # print("avg_runs_horizon: "+str(avg_runs_horizon)+" Is terminal?: "+str(gameLogic.is_state_terminal(next_best_child.state)))
        avg_runs_per_ball = avg_runs_horizon/(self.game_config.horizon_balls - root.state.num_ball)
        
        next_best_child.update_value_of_node(avg_runs_per_ball)
        # print("Simulation phase complete. Runs at horizon:",runs_at_horizon)

        if(not(within_time_constraints(start_time,time_taken_for_loop,action_timeout))):
          simulation_count -= 1
          break

        # Backpropogate node value phase
        value_list = next_best_child.back_propogate_node_value(gameLogic)
        # print("Back propogation phase complete. Value list is: ",value_list)

        current_time =  time. time()
        time_taken_for_loop = current_time - start_of_loop_time

        #Monitor depths reached by simulations for analysis
        all_simulation_depth.append(current_sim_depth)
      
      max_sim_depth = max(all_simulation_depth)
      pos_max_sim__depth = [i for i, j in enumerate(all_simulation_depth) if j == max_sim_depth]
      depth_median = statistics.median(all_simulation_depth)
      depth_mean = statistics.mean(all_simulation_depth)
      current_time =  time. time()
      time_left = (action_timeout - (current_time - start_time))
      print("root node: ")
      print("Act_bat:"+str(root.state.batting_action)+" ValueOfNode:"+str(root.valueOfNode)+" Visit count:"+str(root.visit_count)+" UCB score:"+str(root.ucb_scoreOfNode))
      print("Children of root node: ")
      for each_c  in root.children:
        print("Act_bat:"+str(each_c.state.batting_action)+" ValueOfNode:"+str(each_c.valueOfNode)+" Visit count:"+str(each_c.visit_count)+" UCB score:"+str(each_c.ucb_scoreOfNode))
      print("Current horizon:"+str(current_horizon)+" Current action timeout:"+str(action_timeout))
      print("Total number of simulations: "+str(simulation_count)+" time taken in last loop: "+str(time_taken_for_loop)+" Time left:"+str(time_left) +" Time utilized:"+str(current_time -start_time)+" \n Max depth traversed by simulation: "+str(max_sim_depth)+" simulation position of max depth: "+str(pos_max_sim__depth)+"\n Simulation depth median: "+str(depth_median)+" Simulation depth mean:"+str(depth_mean))
    current_time =  time. time()
    time_left = (overall_action_timeout - (current_time - overall_start_time))
    # print("Root node: "+str(root))
    print("Overall Time left:"+str(time_left) +" Overall Time utilized:"+str(current_time - overall_start_time))

    return self.get_action(gameLogic,root,my_role,simulation_count)

  def get_action(self,gameLogic,root_node,my_role,simulation_count):
    best_child = root_node.select_best_child(gameLogic,round_num = simulation_count)
    if(my_role == 'batting'):
      return best_child.state.batting_action
    elif(my_role == 'bowling'):
      return best_child.state.bowling_action


In [ ]:
class Team:
  def __init__(self,self_features,opponent_features,environment):
    self.self_features = self_features
    self.opponent_features = opponent_features
    self.explore_wicket = 0
    self.explore_runs = 0
    self.environment = environment
    self.current_batters_list =np.array([1,1,1,1,1])  # 0 indicates player who had got out, -1 indicates currently playing bowler in current state, 1 indicates bowler yet to bat
    self.current_bowlers_list =np.array([2,2,2,2,2]) # -number indicates currently playing bowler, +ve number indicates number of overs left

    self.batting_order = [0,1,2,3,4]
    

  # Returns index of player who is not-out as per batting order
  def get_next_batter(self,current_batters_list = None):
    if(current_batters_list is None):
      current_batters_list = self.current_batters_list
    
    for each_order in self.batting_order:
      # If batter is currently batting or already out, skip him
      if(current_batters_list[each_order] > 0):
        return each_order
    
  # For phase 2
  # def get_next_bowler(self):
  #   return np.random.randint(0,5)

  def get_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index,action_timeout,ucb_mode=None,num_rand_rollout_multiplier=None):
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Ball ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^   "+str(ball))
    # batter = -1
    # for index_of_each_full_feature in range(len(self.self_features)):
    #   if((self.self_features[index_of_each_full_feature,0:2] == feature_batter).all()):
    #     batter = index_of_each_full_feature
    ind_array = np.where(self.current_batters_list == -1)[0]
    if(ind_array.shape[0] != 0):
      previous_batter_index = ind_array[0]
      self.current_batters_list[previous_batter_index] = 0
    self.current_batters_list[current_batter_index] = -1

    # bowler = -1
    previous_bowler_index = -1
    # for index_of_each_full_feature in range(len(self.opponent_features)):
    #   if((self.opponent_features[index_of_each_full_feature,2:4] == feature_bowler).all()):
    #     bowler = index_of_each_full_feature
    ind_array = np.where(self.current_bowlers_list < 0)[0]
    if(ball != 0):
      previous_bowler_index = ind_array[0]
      if(ball % 6 == 0):
        self.current_bowlers_list[previous_bowler_index] = (self.current_bowlers_list[previous_bowler_index] + 1)
    
    if(previous_bowler_index == -1 or current_bowler_index != previous_bowler_index):
      self.current_bowlers_list[current_bowler_index] = -(self.current_bowlers_list[current_bowler_index])

    game_config = GameConfig(40,iterative_deepening_depth=4,ucb_mode=ucb_mode,num_rand_rollout_multiplier=num_rand_rollout_multiplier)
    print("get_batting_action Current batters list:",self.current_batters_list)
    print("get_batting_action Current bowlers list:",self.current_bowlers_list)
    mctsAlgo = MCTSAlgo(environment = self.environment,team = self,game_config = game_config)
    current_state = State(ball = ball,total_runs=0,wickets_left = wickets_left,feature_batter=feature_batter,feature_bowler=feature_bowler,current_batters_list=self.current_batters_list,current_bowlers_list=self.current_bowlers_list,batting_action=None,bowling_action=None)
    batting_action = mctsAlgo.run(current_state,action_timeout,my_role='batting')
    
    print("___________Batting action________________: "+str(batting_action))
    
    return batting_action

  def get_bowling_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index):
    bowling_action = np.random.randint(1,4) # this is the place to code UCTS
    return bowling_action
  
  def get_random_batting_action(self,ball,total_runs,wickets_left,score_to_chase,feature_batter,feature_bowler,action_timeout):
    print("~~~~~~~Ball ~~~~~~~~~"+str(ball))
    batting_action = np.random.randint(0,6)  # this is the place to code UCTS
    return batting_action

  def explore(self,explore_num_balls=20):
      """
          Args:---------------------------
          explore_num_balls: int
                            Number of balls provided for testing the capability of 
                            opponent team.
                            (number of balls allower to decide batting order)
          Return:------------------------
          batting order
      """
      balls_per_player = int(explore_num_balls/10)
      batting_features = self.self_features[:,0:2]

      #Run the two scenarios
      results = []
      for case,act_bat,act_bowl,feature_bowler in zip(['Best','Worst'],[6,0],[3,1],np.array([[1,1],[5,5]])):
        result = []
        for j,feature_batter in enumerate(batting_features):
          print(f'Batting: Player {j}')
          wicket = 0
          runs = 0
          for i in range(balls_per_player):
            #print(f'Ball Number: {i+1}')
            w,r = self.environment.get_outcome(feature_batter,feature_bowler,act_bat,act_bowl)
            wickets = wicket+w
            runs =  runs+r
          print(f"Total runs: {runs}")
          result.append(runs) # shape = (5,1)
        results.append(result) # shape = (2,5,1) = (scenario,number of players,runs)
      results = np.array(results)
      # find median scores
      median_scores = np.median(results,0)

      # greedily arrange batting order based on results
      batting_order = np.argsort(median_scores)[::-1]
      self.batting_order = batting_order
      print("Batting order is:"+str(batting_order))
      return batting_order

class Australia(Team):
  def __init__(self,self_features,opponent_features,environment):
    super().__init__(self_features,opponent_features,environment)

class India(Team):
  def __init__(self,self_features,opponent_features,environment):
    super().__init__(self_features,opponent_features,environment)


In [ ]:
"""
WORK IN PROGRESS
a second explore function
"""
def explore(self,explore_num_balls=20):
    """
        Args:---------------------------
        explore_num_balls: int
                          Number of balls provided for testing the capability of 
                          opponent team.
                          (number of balls allower to decide batting order)
        
        feature_batter: int array
                          Features of the batting team, shape = (5,4)
        
        feature_bowler: int array
                          Features of the bowling team, shape = (5,4)


        Return:------------------------
        batting order
    """

    # choose median player
    med_feat_bat = np.array([np.median(self.self_feature[:,0]),np.median(self.self_feature[:,1])]) # shape = (1,1)
    feat_bowl = self.opponent_features[:,2:4] #shape = (5,2)

    #Run the two scenarios
    results = []
    for act_bat,act_bowl in zip([6,0],['a','e']):
      result = []
      for feature_bowler in feat_bowl:
        wickets = 0
        runs = 0
        for i in range(2):
          w,r = self.environment.get_outcome_random(med_feat_bat,feature_bowler,act_bat,act_bowl)
          wickets+=w
          runs+=r
          score = w/2-r/12
        # Record performance of a bowler
        result.append(score)
      #results[0] is the scores from scenario 1 and results[1] is the scores from scenario 2
      results.append(result)  #shape = [2,5,1] = [scenario,player,score]
      
    # find batter order

    
    pass

In [ ]:
class Environment: 
  # def get_outcome_random(self,feature_batter, feature_bowler, batting_action, bowling_action):
  #   wicket = np.random.randint(0,2)
  #   runs = np.random.randint(0,7)
  #   return wicket, runs

  def get_outcome(self,feature_batter, feature_bowler, batting_action, bowling_action,mode='default'):
    """
    Calculate Pout and Pruns. 
    The proportional realations of pruns and pout are calculated using the state values.
    These are then used as a mean value for a normal distribution from which the 
    probablities are sampled. The Variance for each distribution is the same at 0.135.

    Wickets: the value of pout is rounded to the nearest integer (0 or 1).
    Runs: the value of pruns is scaled using batting action then rounded to nearest integer.

    Initial beta distribution is shifted such that avg runs is 3
        Args:-------------------------------------
        feature_batter: batting features of the batter (shape = (2,1))
        feature_bowler: bowling features of the bowler (shape = (2,1))
        batting_action: int between 0 and 6
        bowling_action: int between 1 and 3
        
        
        Returns:----------------------------------
          wickets: int
          runs: int      
    """
    convert = {'e':1,'n':2,'a':3}
    #Define all state values:
    try:
      actbat = batting_action
      actbowl = bowling_action
      avgbat = feature_batter[0]
      sr = feature_batter[1]
      avgbowl = feature_bowler[0]
      eco = feature_bowler[1]
    except:
      print("Error>>>>>>>>>>>>>>>> FBowler:"+str(feature_bowler)+" FBatter: "+str(feature_batter)+" ABowl: "+str(bowling_action)+" Abat:"+str(batting_action))
    
    if isinstance(actbowl,str):
      actbowl = convert[actbowl]

    # Find pruns
    # Define Beta distribution a and b
    mean_runs = actbowl*eco/sr/15
    #mean runs can't ever be 1 if it does n=a=b=0
    mean_runs = min(0.95,mean_runs)
    s = 0.16666667
    n = mean_runs*(1-mean_runs)/s**2
    
    a = mean_runs*n
    if a<=0:
      print(f'a:{a}')
      print(f'n:{n}')
      print(f'mean_runs:{mean_runs}')
      print(f'actbowl:{actbowl}')
    b = n*(1-mean_runs)
    pruns = np.random.beta(a,b)+0.1
    if actbat in [4,6]:
      pruns+=0.1
    if actbat==3:
      pruns-=0.1
    pruns = max(0, min(1, pruns))
    r = int(np.round(actbat*pruns))

    # Find pouts
    # Define a bernoulli dist
    actbat = np.array([1,2,3,4,5,6,6])[actbat]
    mean_outs = (actbat*actbowl*avgbat/avgbowl)/90
    mean_outs = max(0.1, min(1, mean_outs))
    pout = np.random.binomial(1,mean_outs)
    w = int(np.round(pout))

    # check the range of the value of wicket
    w = max(0,min(1,w))
    if r<0:
      r=0
    if r==5:
      r=4
    
    # Remove this code later. Only for testing performance
    # if(actbat == 6):
    #   return 0,5
      
    return w,r
            

class Match:

  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo):
    self.environment = Environment()
    self.num_balls = num_balls
    self.explore_num_balls = explore_num_balls
    self.action_timeout = action_timeout          # time limit for any given act
    self.explore_timeout = explore_timeout        # time limit for exploration phase
    self.team_one_features = np.ones((5,4),dtype=int)   #feature[0] : batting average, feature[1]: strike-rate, feature[2]: bowling average, feature[3]: economy
    self.team_two_features = np.ones((5,4),dtype=int)*5
    self.team_one = TeamOne(self.team_one_features,self.team_two_features,self.environment)
    self.team_two = TeamTwo(self.team_two_features,self.team_one_features,self.environment)
    self.current_batters_list = np.array([1,1,1,1,1])  # a coordinate is set to 0 when that corresponding batter gets out, if the third player is out then you have [1,1,0,1,1]
    self.current_bowlers_list = np.array([2,2,2,2,2])  # if the 4th bowler bowls the first over, after first over we have [2,2,2,1,2].
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0


  def explore_phase_team(self,team_id):    #timed phase
    if (team_id == 1):
      team = self.team_one
    else:
      team = self.team_two
    start_time      = time.time()
    team.explore(self.explore_num_balls)    
    end_time = time.time()    
    if(end_time - start_time > self.explore_timeout):  
      print("Timing Violation During Exploration Phase")
      

  def explore_phase(self):     #allow each of the teams to explore
    print("Team 1 explore")
    self.explore_phase_team(1)
    print("Team 2 explore")
    self.explore_phase_team(2)


  def get_valid_bowler(self,next_bowler):
    if (self.current_bowlers_list[next_bowler]==0):
      print("Bowler Invalid, Choosing Random Bowler")
      bowlers_with_overs_left = np.where(self.current_bowlers_list>0)[0]
      # next_bowler = np.random.choice(bowlers_with_overs_left) # For phase 2
      next_bowler = bowlers_with_overs_left[0]  # Sequentially send bowlers
    return next_bowler


  def get_valid_batter(self,next_batter):
    if (self.current_batters_list[next_batter]==0):
      print("Batter Invalid, Choosing Random Batter"+str(next_batter)+" current_batters_list:"+str(self.current_batters_list))
      batters_not_out = np.where(self.current_batters_list>0)[0]
      next_batter = np.random.choice(batters_not_out)
    return next_batter

  def next_batter(self,team_batting):
    next_batter = team_batting.get_next_batter()
    next_batter = self.get_valid_batter(next_batter)
    feature_batter = team_batting.self_features[next_batter,0:2]
    return next_batter,feature_batter


  def next_bowler(self,team_bowling):
    # next_bowler = team_bowling.get_next_bowler() # For phase 2
    next_bowler = self.get_valid_bowler(0)
    feature_bowler = team_bowling.self_features[next_bowler,2:4]
    return next_bowler, feature_bowler

  def get_team_batting_action(self,team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index,random=False,ucb_mode=None,num_rand_rollout_multiplier=None):
    start_time      = time. time()
    if(random):
      batting_action  = team_batting.get_random_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,self.action_timeout)
    else:
      batting_action  = team_batting.get_batting_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index,self.action_timeout,ucb_mode=ucb_mode,num_rand_rollout_multiplier=num_rand_rollout_multiplier)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      batting_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_batting = self.num_miss_team_batting + 1
      print("Action timeout crossed===================================================================+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
      # raise Exception("Action timeout crossed===================================================================")
    return batting_action


  def get_team_bowling_action(self,team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index):
    start_time      = time. time()
    bowling_action  = team_bowling.get_bowling_action(ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,current_batter_index,current_bowler_index)
    end_time        = time. time()
    if(end_time - start_time > self.action_timeout):
      bowling_action = 0 #this is the default option, we have to fix the penalisation strategy
      self.num_miss_team_bowling = self.num_miss_team_bowling + 1
    return bowling_action



  def innings(self,innigins_id,runs_to_chase,random=False,ucb_mode=None,num_rand_rollout_multiplier=None):
    total_runs = 0
    wickets_left = 5
    self.current_batters_list =np.array([1,1,1,1,1])
    self.current_bowlers_list =np.array([2,2,2,2,2])
    self.num_miss_team_batting = 0
    self.num_miss_team_bowling = 0
    if (innigins_id == 1 ):
      team_batting = self.team_one
      team_bowling = self.team_two
    else:
      team_batting = self.team_two
      team_bowling = self.team_one
    # Initialising the first batter and first bowler
    batter, feature_batter = self.next_batter(team_batting)
    bowler, feature_bowler = self.next_bowler(team_bowling)
    
    for ball in range(self.num_balls):
      if np.sum(self.current_batters_list) > 0 :
        batting_action = self.get_team_batting_action(team_batting,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,batter,bowler,random,ucb_mode=ucb_mode,num_rand_rollout_multiplier=num_rand_rollout_multiplier)
        bowling_action = self.get_team_bowling_action(team_bowling,ball,total_runs,wickets_left,runs_to_chase,feature_batter,feature_bowler,batter,bowler)
        wicket, runs   = self.environment.get_outcome(feature_batter, feature_bowler, batting_action, bowling_action)
        print("Actual score in this ball. Wicket:"+str(wicket)+" run:"+str(runs))
        total_runs     = total_runs + runs
        if (wicket > 0):
          self.current_batters_list[batter] = 0
          wickets_left = wickets_left - 1
          if(np.sum(self.current_batters_list) > 0 ):
            batter,feature_batter = self.next_batter(team_batting)
        if ((ball + 1)%6 ==0 ):
          self.current_bowlers_list[bowler] = self.current_bowlers_list[bowler]-1
          if(np.sum(self.current_bowlers_list) > 0 ) :
            bowler, feature_bowler = self.next_bowler(team_bowling)

    return total_runs, self.current_batters_list, self.current_bowlers_list, self.num_miss_team_batting, self.num_miss_team_bowling


In [11]:

# setting time outs
explore_timeout = 10
action_timeout = 3

explore_num_balls = 60 # this number will be much lesser when we actually test the code.
num_balls = 40         # fixed
match = Match(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India)
match.explore_phase()
first_innings_score_rand, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'),random=True)
print("randomly played Score of innings: "+str(first_innings_score_rand))
first_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
print("Score of innings: "+str(first_innings_score))
print("randomly played Score of innings: "+str(first_innings_score_rand))
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ SECOND INNINGS START @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
# second_innings_score, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(2,first_innings_score)

Team 1 explore
Batting: Player 0
Total runs: 15
Batting: Player 1
Total runs: 17
Batting: Player 2
Total runs: 13
Batting: Player 3
Total runs: 16
Batting: Player 4
Total runs: 12
Batting: Player 0
Total runs: 0
Batting: Player 1
Total runs: 0
Batting: Player 2
Total runs: 0
Batting: Player 3
Total runs: 0
Batting: Player 4
Total runs: 0
Batting order is:[1 3 0 2 4]
Team 2 explore
Batting: Player 0
Total runs: 6
Batting: Player 1
Total runs: 7
Batting: Player 2
Total runs: 9
Batting: Player 3
Total runs: 11
Batting: Player 4
Total runs: 7
Batting: Player 0
Total runs: 0
Batting: Player 1
Total runs: 0
Batting: Player 2
Total runs: 0
Batting: Player 3
Total runs: 0
Batting: Player 4
Total runs: 0
Batting order is:[3 2 4 1 0]
~~~~~~~Ball ~~~~~~~~~0
Actual score in this ball. Wicket:0 run:4
~~~~~~~Ball ~~~~~~~~~1
Actual score in this ball. Wicket:0 run:0
~~~~~~~Ball ~~~~~~~~~2
Actual score in this ball. Wicket:1 run:4
Batter Invalid, Choosing Random Batter1 current_batters_list:[1 0 1 1 1

**Weak opponent Action timeout = 10, ID depth = 5 with act_bat=6=>w=0,r-5**    
Score of innings: 118   
randomly played Score of innings: 110   

Score of innings: 76   
randomly played Score of innings: 124

Score of innings: 132   
randomly played Score of innings: 117   

Score of innings: 92   
randomly played Score of innings: 45  

Score of innings: 93   
randomly played Score of innings: 111    

**Action timeout = 10, ID depth = 10 with act_bat=6=>w=0,r-5**    
Score of innings: 95   
randomly played Score of innings: 133   
   
Score of innings: 133   
randomly played Score of innings: 48   

Score of innings: 108   
randomly played Score of innings: 136   

Score of innings: 130
randomly played Score of innings: 63

**Action Timeout = 10**   
**Strong Opponent**    
random: 134       
RR multiplier:     ['None', '0', '5']    
UCB Modes:None 	[92, 62, 62]    
UCB Modes:VERYLOOSE 	[80, 112, 88]   
UCB Modes:LOOSE 	[126, 136, 145]   
UCB Modes:GREEDY 	[114, 42, 73]   

**Medium Opponent**:    
random 67    
RR multiplier:     ['None', '0', '5']   
UCB Modes:None 	[35, 40, 38]    
UCB Modes:VERYLOOSE 	[69, 69, 77]  
UCB Modes:LOOSE 	[49, 83, 78]   
UCB Modes:GREEDY 	[54, 70, 46]  

**Weak Opponent:**    
random: 19   
RR multiplier:     ['None', '0', '5']   
UCB Modes:None 	[12, 11, 21]   
UCB Modes:VERYLOOSE 	[9, 10, 10]   
UCB Modes:LOOSE 	[21, 9, 6]   
UCB Modes:GREEDY 	[17, 16, 15]   

Strong Opponent random: 66 UCTS: 128  
Medium Opponent random: 36 UCTS: 27    
Weak Opponent random: 11 UCTS: 13  


In [ ]:
def play_innings_with_varied_hyper_parameters(num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo,self_features,opponent_features):
  first_innings_score1_dict = {}
  for (ucb_mode,num_rand_rollout_multiplier) in list(itertools.product([None,'VERYLOOSE','LOOSE','GREEDY'],[None,0,5])):
  # for (ucb_mode,num_rand_rollout_multiplier) in list(itertools.product([None,'VERYLOOSE'],[None,0])):
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("UCB mode:"+str(ucb_mode)+" Multiplier: "+str(num_rand_rollout_multiplier))
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    match = Match2(num_balls, explore_num_balls,action_timeout, explore_timeout,TeamOne,TeamTwo,self_features,opponent_features)
    first_innings_score1, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'),ucb_mode=ucb_mode,num_rand_rollout_multiplier=num_rand_rollout_multiplier)
    if(ucb_mode is None):
      ucb_mode = 'None'
    if(num_rand_rollout_multiplier is None):
      num_rand_rollout_multiplier = 'None'
    else:
      num_rand_rollout_multiplier = str(num_rand_rollout_multiplier)
    if(ucb_mode in first_innings_score1_dict):
      first_innings_score1_dict[ucb_mode][num_rand_rollout_multiplier] = first_innings_score1
    else:
      dict_2 = dict()
      dict_2[num_rand_rollout_multiplier] = first_innings_score1
      first_innings_score1_dict[ucb_mode] = dict_2
  
  return first_innings_score1_dict

In [ ]:
def format_print_results_across_hyper_parameters(dict_1):
  print("RR multiplier:     "+str([each_key for each_key in[each_dict for each_dict in dict_1.values()][0].keys()]))
  for key, dict2 in dict_1.items():
    print("UCB Modes:"+str(key)+" \t"+str([each_value for each_value in dict2.values()]))

In [ ]:
class Match2(Match):
  def __init__(self,num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo,self_features,opponent_features):
    super().__init__(num_balls, explore_num_balls,action_timeout, explore_timeout, TeamOne, TeamTwo)
    self.team_one = TeamOne(self_features,opponent_features,self.environment)
    self.team_two = TeamTwo(opponent_features,self_features,self.environment)


# setting time outs
explore_timeout = 10
action_timeout = 10

explore_num_balls = 60 # this number will be much lesser when we actually test the code.
num_balls = 60         # fixed
# print("Strong Opponent Team: Features All 5 ---------------------------------------------")
# match = Match2(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int),np.ones((5,4),dtype=int)*5)
# match.explore_phase()
# first_innings_score1_rand, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'),random=True)
# print("randomly played Score of innings Strong Opponent Team: "+str(first_innings_score1_rand))
# first_innings_score1_dict = play_innings_with_varied_hyper_parameters(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int),np.ones((5,4),dtype=int)*5)
# print("UCTS played Score of innings Medium Opponent Team: "+str(first_innings_score1))
print()
# print("Medium Opponent Team: Features All 3 ---------------------------------------------")
# match = Match2(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int)*3,np.ones((5,4),dtype=int)*3)
# match.explore_phase()
# first_innings_score2_rand, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'),random=True)
# print("randomly played Score of innings Medium Opponent Team: "+str(first_innings_score2_rand))
# first_innings_score2_dict = play_innings_with_varied_hyper_parameters(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int)*3,np.ones((5,4),dtype=int)*3)
# first_innings_score2, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
# print("UCTS played Score of innings Medium Opponent Team: "+str(first_innings_score2))
# print()
print("Weak Opponent Team: Features All 1 ---------------------------------------------")
match = Match2(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int)*5,np.ones((5,4),dtype=int))
match.explore_phase()
first_innings_score3_rand, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
print("randomly played Score of innings Weak Opponent Team: "+str(first_innings_score3_rand))
first_innings_score3_dict = play_innings_with_varied_hyper_parameters(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int)*5,np.ones((5,4),dtype=int))
# first_innings_score3, batters_list, bowlers_list, num_miss_team_batting, num_miss_team_bowling = match.innings(1,float('inf'))
# print("UCTS played Score of innings Weak Opponent Team: "+str(first_innings_score3))

print('\n')
# print(f"Strong Opponent random: {first_innings_score1_rand}, UCTS: {first_innings_score1_rand}")
# format_print_results_across_hyper_parameters(first_innings_score1_dict)
# print(f"Medium Opponent random: {first_innings_score2_rand} UCTS: {first_innings_score2_rand}")
# format_print_results_across_hyper_parameters(first_innings_score2_dict)
print(f"Weak Opponent random: {first_innings_score3_rand}")
format_print_results_across_hyper_parameters(first_innings_score3_dict)

#first_innings_score1_dict = play_innings_with_varied_hyper_parameters(num_balls, explore_num_balls,action_timeout, explore_timeout,Australia,India,np.ones((5,4),dtype=int),np.ones((5,4),dtype=int)*5)
#format_print_results_across_hyper_parameters(first_innings_score1_dict)


Weak Opponent Team: Features All 1 ---------------------------------------------
Team 1 explore
Batting: Player 0
Total runs: 6
Batting: Player 1
Total runs: 9
Batting: Player 2
Total runs: 6
Batting: Player 3
Total runs: 8
Batting: Player 4
Total runs: 6
Batting: Player 0
Total runs: 0
Batting: Player 1
Total runs: 0
Batting: Player 2
Total runs: 0
Batting: Player 3
Total runs: 0
Batting: Player 4
Total runs: 0
Batting order is:[1 3 4 2 0]
Team 2 explore
Batting: Player 0
Total runs: 16
Batting: Player 1
Total runs: 19
Batting: Player 2
Total runs: 15
Batting: Player 3
Total runs: 15
Batting: Player 4
Total runs: 13
Batting: Player 0
Total runs: 0
Batting: Player 1
Total runs: 0
Batting: Player 2
Total runs: 0
Batting: Player 3
Total runs: 0
Batting: Player 4
Total runs: 0
Batting order is:[1 0 3 2 4]
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Ball ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^   0
get_batting_action Current batters list: [ 1 -1  1  1  1]
get_batting_action Current bowlers list: 

In [ ]:
"""
      --------------------------------------------- TESTING get_outcome----------------------------------------------
"""
# Exhaustive search of values of wickets and runs
env = Environment()
wicket = []
runs = []
for actbat in range(1,7):
  if actbat ==5:
    continue
  for actbowl in range(1,4):
    for avgbowl in range(1,6):
      for avgbat in range(1,6):
        for eco in range(1,6):
          for sr in range(1,6):
            w,r = env.get_outcome(np.array([avgbat,sr]),np.array([avgbowl,eco]),actbat,actbowl)    
            wicket.append(w)       
            runs.append(r)

print("GET OUTCOME METHOD PERFORMANCE")
print(f'bin count for wickets: {np.bincount(np.array(wicket))}')
print(f'bin count for runs: {np.bincount(np.array(runs))}')

"""
-----------------------------------Testing Explore Method------------------------------
"""
team = Team(np.random.uniform(1,5,size=(5,4)),np.random.uniform(1,5,size=(5,4)),env)
batting_order = team.explore()

In [ ]:
np.ones((5,4),dtype=int)*5